In [1]:

# from __future__ import unicode_literals, print_function, division
# from io import open
# ======= features from python 2.x, no need here ========

import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# from bpe import BPETokenizer
# from imp import reload
# reload(bpe)
# tokenizer = BPETokenizer()
# # BPETokenizer()(text='我操你的').numpy().tolist()[0]
# tokenizer(text='開')[0]

In [2]:
# encoding words

SOS_token = 0
EOS_token = 1
UNK_token = 2

# old Land: count based
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"<UNK>": UNK_token}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "<UNK>"}
        self.n_words = 3  # Count SOS and EOS + UNK

    def addSentence(self, sentence):

        for word in sentence.split(' '):
            self.addWord(word)


    def addWord(self, word):

        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1

        else:
            self.word2count[word] += 1




# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

# 为了更好地兼容性，把normalizeString改成normalize_english，实际二者的处理逻辑完全一致
def normalize_english(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

def normalize_chinese(s):
    s = s.strip()
    s = ' '.join(list(s))  # 字符级别分割
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

# 不把原始中文文本用空格分开，因为现在用了BPETokenizer，分词不需要再依赖空格
# def normalize_chinese(s):
#     s = s.strip()
#     s = get_enc(s, e)
#     return s.strip()

In [3]:
# read data (fra2eng)
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('kaggleData/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

# read data (cn2eng)
def read_langs(lang1, lang2, reverse=False):
    # lines = open('data/cmn.txt', encoding='utf-8').read().split('\n')
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('kaggleData/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    
    pairs = []
    for line in lines:
        parts = line.split('\t')
        if len(parts) < 2: continue
        en = normalize_english(parts[0].strip())
        zh = normalize_chinese(parts[1].strip())
        pairs.append([zh, en] if reverse else [en, zh])
    input_lang = Lang('cmn') if reverse else Lang('eng')
    output_lang = Lang('eng') if reverse else Lang('cmn')
    return input_lang, output_lang, pairs

In [4]:
# filter data, extract short sentence
# MAX_LENGTH = 45: the minimal length that can use the whole data set without being trimmed
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH 

# def filterPair(p):
#     return p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    # input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    input_lang, output_lang, pairs = read_langs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
input_lang, output_lang, pairs = prepareData('eng', 'cmn', True)
print(random.choice(pairs))

print(input_lang.n_words)



Reading lines...
Read 29909 sentence pairs
Trimmed to 1563 sentence pairs
Counting words...
Counted words:
cmn 1170
eng 1084
['他 有 点 活 泼 。', 'he s a bit lively']
1170


In [12]:
pairs

[['我 沒 事 。', 'i m ok'],
 ['我 已 经 起 来 了 。', 'i m up'],
 ['我 觉 得 很 热 。', 'i m hot'],
 ['我 生 病 了 。', 'i m ill'],
 ['我 很 难 过 。', 'i m sad'],
 ['我 很 害 羞 。', 'i m shy'],
 ['我 濕 了 。', 'i m wet'],
 ['我 回 来 了 ！', 'i m back'],
 ['我 很 忙 。', 'i m busy'],
 ['我 冷 。', 'i m cold'],
 ['我 很 酷 。', 'i m cool'],
 ['我 很 好 。', 'i m fine'],
 ['我 自 由 了 ！', 'i m free !'],
 ['我 吃 飽 了 。', 'i m full'],
 ['我 在 家 裡 。', 'i m home'],
 ['我 迷 失 了 。', 'i m lost'],
 ['我 生 病 了 。', 'i m sick'],
 ['我 病 了 。', 'i m sick'],
 ['我 个 子 高 。', 'i m tall'],
 ['他 是 一 个 D J 。', 'he s a dj'],
 ['他 很 忙 。', 'he s busy'],
 ['他 很 懒 。', 'he s lazy'],
 ['我 冷 。', 'i am cold'],
 ['我 吃 飽 了 。', 'i am full'],
 ['我 沒 事 。', 'i am okay'],
 ['我 生 病 了 。', 'i am sick'],
 ['我 个 子 高 。', 'i am tall'],
 ['我 是 个 男 人 。', 'i m a man'],
 ['我 覺 得 很 無 聊 。', 'i m bored'],
 ['我 是 第 一 。', 'i m first'],
 ['我 运 气 很 好 。', 'i m lucky'],
 ['我 是 對 的 。', 'i m right'],
 ['我 很 清 醒 。', 'i m sober'],
 ['对 不 起 。', 'i m sorry'],
 ['我 很 抱 歉 。', 'i m sorry'],
 ['我 很 累 。', 'i m tir

In [5]:
# define encode-decoder rnn
# notice that the DecoderRNN here is not attn version, just for demo, will not be used later

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden
    

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [6]:
# introduce decoder with attn

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [7]:
# prepare data from training
def indexesFromSentence(lang, sentence):
    return [lang.word2index.get(word, UNK_token) for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    # input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
    input_lang, output_lang, pairs = prepareData('eng', 'cmn', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    print(pairs[:2])
    print(target_ids[:2])
    print(input_ids[:2])
    return input_lang, output_lang, train_dataloader

# def get_dataloader(batch_size):
#     # input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#     input_lang, output_lang, pairs = prepareData('eng', 'cmn', file_path,True)

#     n = len(pairs)
#     input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
#     target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

#     for idx, (inp, tgt) in enumerate(pairs):
#         inp_ids = indexesFromSentence(input_lang, inp)
#         tgt_ids = indexesFromSentence(output_lang, tgt)
#         inp_ids.append(EOS_token)
#         tgt_ids.append(EOS_token)
#         input_ids[idx, :len(inp_ids)] = inp_ids
#         target_ids[idx, :len(tgt_ids)] = tgt_ids

#     # ============== original =======================
#     # train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
#     #                            torch.LongTensor(target_ids).to(device))

#     # train_sampler = RandomSampler(train_data)
#     # train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
#     # ===============================================

#     dataset = TensorDataset(torch.LongTensor(input_ids).to(device),
#                                torch.LongTensor(target_ids).to(device))
#     train_data, test_data = random_split(dataset = dataset, lengths = [0.9,0.1])
#     train_dataloader = DataLoader(train_data,  batch_size=batch_size)
#     test_dataloader = DataLoader(test_data,  batch_size=batch_size)


#     # print(pairs[:2])
#     # print(target_ids[:2])
#     # print(input_ids[:2])
#     return input_lang, output_lang, train_dataloader, test_dataloader

In [8]:
# training misc: plot print etc

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [9]:
# eval 

def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn


def evaluateRandomly(encoder, decoder, n=100):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [10]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 29909 sentence pairs
Trimmed to 1563 sentence pairs
Counting words...
Counted words:
cmn 1170
eng 1084
[['我 沒 事 。', 'i m ok'], ['我 已 经 起 来 了 。', 'i m up']]
[[3 4 5 1 0 0 0 0 0 0]
 [3 4 6 1 0 0 0 0 0 0]]
[[ 3  4  5  6  1  0  0  0  0  0]
 [ 3  7  8  9 10 11  6  1  0  0]]
0m 19s (- 4m 59s) (5 6%) 2.4232


KeyboardInterrupt: 

In [11]:
train_dataloader.dataset[0]

(tensor([3, 4, 5, 6, 1, 0, 0, 0, 0, 0]),
 tensor([3, 4, 5, 1, 0, 0, 0, 0, 0, 0]))

In [11]:
# save model
torch.save(encoder.state_dict(), "encoder_weights.pth")
torch.save(decoder.state_dict(), "decoder_weights.pth")

In [12]:
encoder.load_state_dict(torch.load('encoder_weights.pth', map_location=device))
decoder.load_state_dict(torch.load('decoder_weights.pth', map_location=device))
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> 很 抱 歉 我 伤 害 了 你 。
= i m sorry that i hurt you
< i m sorry that i hurt you <EOS>

> 他 習 慣 了 旅 行 。
= he s accustomed to traveling
< he s accustomed to traveling <EOS>

> 她 现 在 正 在 吃 午 餐 。
= she is having lunch now
< she is having lunch now <EOS>

> 你 開 玩 笑 吧 ！
= you re kidding !
< you re kidding ! <EOS>

> 他 很 高 。
= he is very tall
< he is very tall <EOS>

> 我 单 身 。
= i am not married
< i m a bachelor <EOS>

> 他 真 是 甜 心 。
= he s such a sweetheart
< he s such a sweetheart <EOS>

> 他 是 家 里 的 宝 贝 。
= he is the baby of the family
< he is the baby of the family <EOS>

> 你 總 是 遲 到 。
= you re always late
< you are always late <EOS>

> 我 很 寂 寞 。
= i m very lonely
< i m very lonely <EOS>

> 他 现 在 出 去 了 。
= he s out now
< he s out now <EOS>

> 我 们 是 特 别 的 。
= we re special
< we re special <EOS>

> 我 们 谁 都 不 怕 。
= we re not scared of anybody
< we re not scared of anybody <EOS>

> 我 是 個 輸 家 。
= i m a loser
< i m a loser <EOS>

> 他 跑 得 非 常 快 。
= he s running very quickly
< he s running very quickly

In [238]:
# 待翻译法语句子
french_sentence = "je suis malade"

# 完整翻译过程
def translate(sentence):
    # 1. 文本标准化
    normalized = normalizeString(sentence)
    
    # 2. 转换为模型输入张量
    input_tensor = tensorFromSentence(input_lang, normalized)
    
    # 3. 模型推理
    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)
    
    # 4. 获取预测结果
    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()
    
    # 5. 转换为英文单词
    translated_words = []
    for idx in decoded_ids:
        if idx.item() == EOS_token:
            break
        translated_words.append(output_lang.index2word[idx.item()])
    
    return ' '.join(translated_words)

print(translate(french_sentence))  # 输出：she is five years younger than me .

i m sick
